# DCMN Explore Notebook

## Step 1: Load data and see what our problems are like!

In [1]:
import os
from data import read_examples

data_path = './RACE/'
train_high, train_middle = read_examples(os.path.join(data_path, 'train'))

In [2]:
from utils import print_example

print_example(train_high, 0)

id: 
 high1000.txt
passage: 
 When newspapers and radio describe the damage caused by a hurricane  named Hazel, girls named Hazel are probably teased  by their friends. To keep out of trouble, the Weather Bureau says,"Any _ between hurricane names and the names of particular girls is purely accidental."
Some women became angry because hurricanes are given their names, but many other women are proud to see their names make headlines. They don't even care that they are the names of destructive storms. Because more women seem to like it than dislike it, the Weather Bureau has decided to continue using girl's names for hurricanes.
In some ways a hurricane is like a person. After it is born, it grows and develops, then becomes old and dies. Each hurricane has a character of its own. Each follows its own path through the world, and people remember it long after it gone. So it is natural to give hurricanes' names, and to talk about them almost if they were alive.
questions: 
 ['What happens t

## Step 2: Getting a BERT layer and its corresponding preprocessor
We found bert-base to be too big for our hardware, so two smaller bert instances are tried. Among those the 2-layer-128-hidden one also only has a `max-seq-length` of 128, instead of the original 512. Considering the length of our passages a larger small model is used instead.

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

arguments = dict(seq_length=512)

#preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_preprocess = hub.KerasLayer(preprocess)

#bert = hub.load('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1')
bert = hub.load('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/2')
bert_layer = hub.KerasLayer(bert)

In [18]:
from preprocessing import dcmn_preprocess

sample_size = 1000
sample_data = train_high[:1000]
validation_data = train_high[sample_size:sample_size + 100]

sample_inputs, sample_answers = dcmn_preprocess(sample_data)
valid_inputs, valid_answers = dcmn_preprocess(validation_data)
answer_dict = {'A': tf.constant([1, 0, 0, 0]),
               'B': tf.constant([0, 1, 0, 0]),
               'C': tf.constant([0, 0, 1, 0]),
               'D': tf.constant([0, 0, 0, 1])}
sample_answers_encoded = tf.convert_to_tensor([answer_dict[a] for a in sample_answers])
valid_answers_encoded = tf.convert_to_tensor([answer_dict[a] for a in valid_answers])

## Compiling the model and training!

In [7]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense
from model import DCMN, GateLayers

config = {'n_choices': 4,
          'hidden_size': 256, #Under current setup, must adjust to match bert embedding size
          'dropout': 0.1,
          'bert_preprocess': bert_preprocess,
          'bert': bert_layer,
          'gate_layer': GateLayers,
          'classifier': Dense(4, activation='softmax')}

model = DCMN(config)

In [8]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(5e-3),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = [tf.keras.metrics.CategoricalAccuracy()]
)

model.fit(x = sample_inputs, y = sample_answers_encoded, batch_size = 10, epochs = 5, validation_data=(valid_inputs, valid_answers_encoded))

Epoch 1/5
325/325 [==============================] - 69s 193ms/step - loss: 1.7241 - categorical_accuracy: 0.2510 - val_loss: 1.4667 - val_categorical_accuracy: 0.3059
Epoch 2/5
325/325 [==============================] - 63s 194ms/step - loss: 1.6140 - categorical_accuracy: 0.2818 - val_loss: 1.5714 - val_categorical_accuracy: 0.2895
Epoch 3/5
325/325 [==============================] - 62s 191ms/step - loss: 1.6178 - categorical_accuracy: 0.2932 - val_loss: 1.5602 - val_categorical_accuracy: 0.2982
Epoch 4/5
325/325 [==============================] - 63s 193ms/step - loss: 1.6365 - categorical_accuracy: 0.3068 - val_loss: 1.4048 - val_categorical_accuracy: 0.3478
Epoch 5/5
325/325 [==============================] - 64s 198ms/step - loss: 1.5418 - categorical_accuracy: 0.3090 - val_loss: 1.3041 - val_categorical_accuracy: 0.3932


In [9]:
model.fit(x = sample_inputs, y = sample_answers_encoded, batch_size = 10, epochs = 10, validation_data=(valid_inputs, valid_answers_encoded))

Epoch 1/10
325/325 [==============================] - 64s 195ms/step - loss: 1.4691 - categorical_accuracy: 0.3361 - val_loss: 1.4420 - val_categorical_accuracy: 0.3771
Epoch 2/10
325/325 [==============================] - 64s 196ms/step - loss: 1.4840 - categorical_accuracy: 0.3315 - val_loss: 1.4612 - val_categorical_accuracy: 0.3605
Epoch 3/10
325/325 [==============================] - 62s 192ms/step - loss: 1.4710 - categorical_accuracy: 0.3447 - val_loss: 1.6720 - val_categorical_accuracy: 0.3534
Epoch 4/10
325/325 [==============================] - 61s 189ms/step - loss: 1.5667 - categorical_accuracy: 0.3358 - val_loss: 1.3973 - val_categorical_accuracy: 0.4049
Epoch 5/10
325/325 [==============================] - 64s 196ms/step - loss: 1.5294 - categorical_accuracy: 0.3549 - val_loss: 1.3826 - val_categorical_accuracy: 0.3938
Epoch 6/10
325/325 [==============================] - 62s 191ms/step - loss: 1.5824 - categorical_accuracy: 0.3410 - val_loss: 1.3140 - val_categorical_acc

In [10]:
model.fit(x = sample_inputs, y = sample_answers_encoded, batch_size = 10, epochs = 20, validation_data=(valid_inputs, valid_answers_encoded))

Epoch 1/20
325/325 [==============================] - 63s 193ms/step - loss: 1.5176 - categorical_accuracy: 0.3626 - val_loss: 1.2669 - val_categorical_accuracy: 0.4425
Epoch 2/20
325/325 [==============================] - 61s 189ms/step - loss: 1.4682 - categorical_accuracy: 0.3682 - val_loss: 1.3286 - val_categorical_accuracy: 0.4289
Epoch 3/20
325/325 [==============================] - 61s 189ms/step - loss: 1.4560 - categorical_accuracy: 0.3910 - val_loss: 1.1997 - val_categorical_accuracy: 0.4678
Epoch 4/20
325/325 [==============================] - 62s 191ms/step - loss: 1.4656 - categorical_accuracy: 0.3885 - val_loss: 1.6048 - val_categorical_accuracy: 0.3549
Epoch 5/20
325/325 [==============================] - 63s 193ms/step - loss: 1.4793 - categorical_accuracy: 0.3679 - val_loss: 1.3531 - val_categorical_accuracy: 0.4258
Epoch 6/20
325/325 [==============================] - 62s 191ms/step - loss: 1.3939 - categorical_accuracy: 0.4012 - val_loss: 1.3491 - val_categorical_acc

In [11]:
model.fit(x = sample_inputs, y = sample_answers_encoded, batch_size = 10, epochs = 20, validation_data=(valid_inputs, valid_answers_encoded))

Epoch 1/20
325/325 [==============================] - 60s 184ms/step - loss: 1.3414 - categorical_accuracy: 0.4564 - val_loss: 1.2453 - val_categorical_accuracy: 0.4820
Epoch 2/20
325/325 [==============================] - 59s 182ms/step - loss: 1.3399 - categorical_accuracy: 0.4533 - val_loss: 1.2465 - val_categorical_accuracy: 0.4915
Epoch 3/20
325/325 [==============================] - 60s 185ms/step - loss: 1.3688 - categorical_accuracy: 0.4342 - val_loss: 1.1547 - val_categorical_accuracy: 0.5088
Epoch 4/20
325/325 [==============================] - 59s 181ms/step - loss: 1.3388 - categorical_accuracy: 0.4453 - val_loss: 1.1918 - val_categorical_accuracy: 0.4968
Epoch 5/20
325/325 [==============================] - 59s 181ms/step - loss: 1.3468 - categorical_accuracy: 0.4576 - val_loss: 1.2611 - val_categorical_accuracy: 0.4727
Epoch 6/20
325/325 [==============================] - 59s 181ms/step - loss: 1.3794 - categorical_accuracy: 0.4419 - val_loss: 1.3383 - val_categorical_acc

In [16]:
model.fit(x = sample_inputs, y = sample_answers_encoded, batch_size = 10, epochs = 20, validation_data=(valid_inputs, valid_answers_encoded))

Epoch 1/20
325/325 [==============================] - 58s 177ms/step - loss: 1.1895 - categorical_accuracy: 0.5211 - val_loss: 1.4590 - val_categorical_accuracy: 0.4357
Epoch 2/20
325/325 [==============================] - 63s 194ms/step - loss: 1.2842 - categorical_accuracy: 0.5042 - val_loss: 1.2659 - val_categorical_accuracy: 0.4878
Epoch 3/20
325/325 [==============================] - 63s 194ms/step - loss: 1.2598 - categorical_accuracy: 0.4980 - val_loss: 1.1062 - val_categorical_accuracy: 0.5424
Epoch 4/20
325/325 [==============================] - 62s 192ms/step - loss: 1.2234 - categorical_accuracy: 0.5109 - val_loss: 1.5501 - val_categorical_accuracy: 0.4132
Epoch 5/20
325/325 [==============================] - 63s 193ms/step - loss: 1.2415 - categorical_accuracy: 0.5005 - val_loss: 1.0561 - val_categorical_accuracy: 0.5646
Epoch 6/20
325/325 [==============================] - 62s 191ms/step - loss: 1.1548 - categorical_accuracy: 0.5378 - val_loss: 1.3105 - val_categorical_acc

In [17]:
chkpt_path = './trained_models/dcmn_bert_4_256_epoch_80/'
model.save_weights(chkpt_path)